In [83]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configurar o WebDriver do Selenium
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=chrome_options)

# URL do site para extrair os dados
url = "https://www.linkedin.com/jobs/search?keywords=%28%22%28data%20science%29%22%20OR%20%22cientista%20de%20dados%29%22%20OR%20%22machine%20learning%22%29&location=Brasil&geoId=106057199&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0"

# Lista para armazenar os dados extraídos
data = []

# Acessar a página
driver.get(url)
wait = WebDriverWait(driver, 10)

# Tempo de espera para carregar mais trabalhos (ajuste conforme necessário)
wait_time = 3


# Rolar a página até o final máximo possível
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(wait_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    
# Verificar se há o botão "Ver mais vagas" e clicar nele até 30 vezes, se necessário
click_count = 0
max_clicks = 10
while click_count < max_clicks:
    show_more_jobs_button = driver.find_element(By.CSS_SELECTOR, ".infinite-scroller__show-more-button")
    if show_more_jobs_button.is_displayed():
        show_more_jobs_button.click()
        time.sleep(wait_time)
        click_count += 1
        print(click_count)
    else:
        break

# Extrair os elementos de trabalho da página
soup = BeautifulSoup(driver.page_source, "html.parser")
job_elements = soup.select(".base-card__full-link")

# Extrair os dados de cada trabalho
for element in job_elements:
    # Verificar se o elemento possui o atributo href
    if "href" not in element.attrs:
        continue

    # Obter o link do trabalho
    link = element["href"]

    # Clicar no elemento para abrir a vaga na página dinâmica
    driver.get(link)
    time.sleep(wait_time)

    # Extrair o título do trabalho
    title_element = driver.find_element(By.CSS_SELECTOR, ".top-card-layout__title")
    if title_element is not None:
        title = title_element.text.strip()
    else:
        title = ""

    # Extrair o nome da empresa
    company_element = driver.find_element(By.CSS_SELECTOR, ".topcard__org-name-link")
    if company_element is not None:
        company = company_element.text.strip()
    else:
        company = ""

    # Extrair a localização
    location_element = driver.find_element(By.CSS_SELECTOR, ".topcard__flavor--bullet")
    if location_element is not None:
        location = location_element.text.strip()
    else:
        location = ""

    # Clicar no botão "Exibir mais" para obter a descrição expandida
    show_more_button = driver.find_element(By.CSS_SELECTOR, ".show-more-less-html__button")
    if show_more_button is not None:
        show_more_button.click()
        time.sleep(wait_time)

        # Extrair a descrição expandida
        expanded_description_element = driver.find_element(By.CSS_SELECTOR, ".show-more-less-html__markup")
        if expanded_description_element is not None:
            expanded_description = expanded_description_element.text.strip()
        else:
            expanded_description = ""
    else:
        expanded_description = ""

    # Verificar se o trabalho já está na lista de dados
    job_id = link.split("/")[-1].split("?")[0]
    if any(job_id == item["ID"] for item in data):
        continue

    # Adicionar os dados à lista
    data.append({
        "ID": job_id,
        "DATA": pd.Timestamp.now().strftime("%Y-%m-%d"),
        "TITULO": title,
        "EMPRESA": company,
        "LOCAL": location,
        "LINK": link,
        "DESCRICAO": expanded_description
    })

    # Converter a lista de dados em um DataFrame do Pandas
    df = pd.DataFrame(data)
    
    # Exibir uma amostra do DataFrame
    print(df.sample())

    # Salvar os dados em um arquivo CSV
    df.to_csv("linkedin_jobs.csv", index=False)

# Exibir uma amostra do DataFrame
print(df.sample(5))

# Fechar o WebDriver
driver.quit()


1
2
3
4
5
6
7
8
9
10
                                                  ID        DATA  \
0  lighthouse-programa-de-forma%C3%A7%C3%A3o-em-d...  2023-07-11   

                                              TITULO   EMPRESA  \
0  Lighthouse - Programa de formação em dados (Re...  Indicium   

               LOCAL                                               LINK  \
0  Florianópolis, SC  https://br.linkedin.com/jobs/view/lighthouse-p...   

                                           DESCRICAO  
0  O Lighthouse é um programa da Indicium para es...  
                                                  ID        DATA  \
0  lighthouse-programa-de-forma%C3%A7%C3%A3o-em-d...  2023-07-11   

                                              TITULO   EMPRESA  \
0  Lighthouse - Programa de formação em dados (Re...  Indicium   

               LOCAL                                               LINK  \
0  Florianópolis, SC  https://br.linkedin.com/jobs/view/lighthouse-p...   

                           

                                                  ID        DATA  \
1  cientista-criativo-est%C3%A1gio-at-winnin-3623...  2023-07-11   

                         TITULO EMPRESA               LOCAL  \
1  CIENTISTA CRIATIVO | ESTÁGIO  Winnin  Rio de Janeiro, RJ   

                                                LINK  \
1  https://br.linkedin.com/jobs/view/cientista-cr...   

                                           DESCRICAO  
1  Sobre a Winnin\n\nO Winnin Insights é uma plat...  
                                                   ID        DATA  \
10  pessoa-cientista-de-dados-s%C3%AAnior-llm-at-a...  2023-07-11   

                                    TITULO EMPRESA               LOCAL  \
10  Pessoa Cientista de Dados Sênior (LLM)  A3Data  Belo Horizonte, MG   

                                                 LINK  \
10  https://br.linkedin.com/jobs/view/pessoa-cient...   

                                            DESCRICAO  
10  A A3Data é uma consultoria de dados, pioneira ... 

                                                   ID        DATA  \
14  gerente-executivo-a-de-modelagem-de-cr%C3%A9di...  2023-07-11   

                                           TITULO    EMPRESA          LOCAL  \
14  Gerente Executivo (a) de Modelagem de Crédito  Banco PAN  São Paulo, SP   

                                                 LINK  \
14  https://br.linkedin.com/jobs/view/gerente-exec...   

                                            DESCRICAO  
14  Quem é o PAN ?\n\nSomos uma das principais ins...  
                                                  ID        DATA  \
3  pessoa-engenheira-de-dados-junior-at-clickbus-...  2023-07-11   

                              TITULO   EMPRESA   LOCAL  \
3  PESSOA ENGENHEIRA DE DADOS JUNIOR  ClickBus  Brasil   

                                                LINK  \
3  https://br.linkedin.com/jobs/view/pessoa-engen...   

                                           DESCRICAO  
3  O que nos move diariamente?\n\nTodos os dias v... 

                                                   ID        DATA  \
26  analista-customer-science-sr-at-banco-bv-36547...  2023-07-11   

                          TITULO   EMPRESA          LOCAL  \
26  Analista Customer Science Sr  banco BV  São Paulo, SP   

                                                 LINK  \
26  https://br.linkedin.com/jobs/view/analista-cus...   

                                            DESCRICAO  
26  Sobre o banco BV\n\nUm banco nacional, o 5º ma...  
                                                 ID        DATA  \
7  senior-data-scientist-at-sword-health-3651752184  2023-07-11   

                  TITULO       EMPRESA              LOCAL  \
7  Senior Data Scientist  Sword Health  Curitiba e Região   

                                                LINK  \
7  https://br.linkedin.com/jobs/view/senior-data-...   

                                           DESCRICAO  
7  Who We Are\n\nEach year, 2 billion people acro...  
                              

                                                   ID        DATA  \
65  analista-pleno-de-b-i-at-venture-people-363266...  2023-07-11   

                    TITULO         EMPRESA          LOCAL  \
65  Analista Pleno de B.I.  Venture People  São Paulo, SP   

                                                 LINK  \
65  https://br.linkedin.com/jobs/view/analista-ple...   

                                            DESCRICAO  
65  O Adiante está à procura de um Analista Pleno ...  
                                                   ID        DATA  \
65  analista-pleno-de-b-i-at-venture-people-363266...  2023-07-11   

                    TITULO         EMPRESA          LOCAL  \
65  Analista Pleno de B.I.  Venture People  São Paulo, SP   

                                                 LINK  \
65  https://br.linkedin.com/jobs/view/analista-ple...   

                                            DESCRICAO  
65  O Adiante está à procura de um Analista Pleno ...  
                      

...

                                                 ID        DATA  \
7  senior-data-scientist-at-sword-health-3651752184  2023-07-11   

                  TITULO       EMPRESA              LOCAL  \
7  Senior Data Scientist  Sword Health  Curitiba e Região   

                                                LINK  \
7  https://br.linkedin.com/jobs/view/senior-data-...   

                                           DESCRICAO  
7  Who We Are\n\nEach year, 2 billion people acro...  
                                                    ID        DATA  \
329  analytical-roles-%E2%80%93-analyst-senior-anal...  2023-07-11   

                                                TITULO EMPRESA  \
329  Analytical roles – Analyst/ Senior Analyst/ As...   Agoda   

                  LOCAL                                               LINK  \
329  Rio de Janeiro, RJ  https://br.linkedin.com/jobs/view/analytical-r...   

                                             DESCRICAO  
329  About Agoda\n\nAgoda is an

                                            ID        DATA  \
34  pessoa-agilista-ii-at-neurotech-3651719125  2023-07-11   

                TITULO    EMPRESA       LOCAL  \
34  Pessoa Agilista II  Neurotech  Recife, PE   

                                                 LINK  \
34  https://br.linkedin.com/jobs/view/pessoa-agili...   

                                            DESCRICAO  
34  CHEGOU SUA HORA DE BRILHAR!\n\n💎Você quer se t...  
                                                    ID        DATA  \
132  auditor-a-interno-pleno-at-ra%C3%ADzen-3623013678  2023-07-11   

                       TITULO EMPRESA           LOCAL  \
132  Auditor(a) Interno Pleno  Raízen  Piracicaba, SP   

                                                  LINK  \
132  https://br.linkedin.com/jobs/view/auditor-a-in...   

                                             DESCRICAO  
132  Job Description\n\nEstá preparado(a) para cult...  
                                                   ID        D

                                                    ID        DATA  \
230  principal-clinical-data-manager-latam-iqvia-bi...  2023-07-11   

                                                TITULO EMPRESA          LOCAL  \
230  Principal Clinical Data Manager, LATAM - IQVIA...   IQVIA  São Paulo, SP   

                                                  LINK  \
230  https://br.linkedin.com/jobs/view/principal-cl...   

                                             DESCRICAO  
230  Principal Data Manager\n\nJob Overview\n\nPrin...  
                                                    ID        DATA  \
259  senior-data-scientist-poatek-brazil-applicants...  2023-07-11   

                                                TITULO     EMPRESA  \
259  Senior Data Scientist (Poatek- Brazil Applicants)  WillowTree   

                LOCAL                                               LINK  \
259  Porto Alegre, RS  https://br.linkedin.com/jobs/view/senior-data-...   

                              

                                       ID        DATA                 TITULO  \
5  cientista-de-dados-sr-at-rd-3655205346  2023-07-11  Cientista de Dados Sr   

  EMPRESA              LOCAL  \
5      RD  São Paulo, Brasil   

                                                LINK  \
5  https://br.linkedin.com/jobs/view/cientista-de...   

                                           DESCRICAO  
5  O grupo RD – Gente, Saúde e Bem-estar é uma em...  
                                                  ID        DATA  \
51  ger-planejamento-demanda-jr-at-danone-3654362439  2023-07-11   

                         TITULO EMPRESA          LOCAL  \
51  GER PLANEJAMENTO DEMANDA JR  Danone  São Paulo, SP   

                                                 LINK  \
51  https://br.linkedin.com/jobs/view/ger-planejam...   

                                            DESCRICAO  
51  About The Job\n\nLiderar o Planejamento de Dem...  
                                                   ID        DATA  \
4

                                                    ID        DATA  \
312  analytical-roles-%E2%80%93-analyst-senior-anal...  2023-07-11   

                                                TITULO EMPRESA  \
312  Analytical roles – Analyst/ Senior Analyst/ As...   Agoda   

                  LOCAL                                               LINK  \
312  São Paulo e Região  https://br.linkedin.com/jobs/view/analytical-r...   

                                             DESCRICAO  
312  About Agoda\n\nAgoda is an online travel booki...  
                                                    ID        DATA  \
262  arquiteto-de-dados-aws-s%C3%AAnior-at-leega-36...  2023-07-11   

                            TITULO EMPRESA          LOCAL  \
262  Arquiteto de dados AWS Sênior   Leega  São Paulo, SP   

                                                  LINK  \
262  https://br.linkedin.com/jobs/view/arquiteto-de...   

                                             DESCRICAO  
262  Responsabili

In [86]:
len(df)

411

In [87]:
df.sample(10)

,ID,DATA,TITULO,EMPRESA,LOCAL,LINK,DESCRICAO
274,software-engineering-manager-at-adp-brazil-lab...,2023-07-11,Software Engineering Manager,ADP Brazil Labs,"Porto Alegre, RS",https://br.linkedin.com/jobs/view/software-eng...,ADP SBS division is hiring a Manager Product D...
119,remote-ad-optimizer-latam-at-monetizemore-3659...,2023-07-11,Remote Ad Optimizer - LATAM,MonetizeMore,"Curitiba, PR",https://br.linkedin.com/jobs/view/remote-ad-op...,MonetizeMore is a global leader in ad revenue ...
399,part-time-personalized-internet-ads-assessor-g...,2023-07-11,Part-Time Personalized Internet Ads Assessor -...,TELUS International AI Data Solutions,Brasil,https://br.linkedin.com/jobs/view/part-time-pe...,Company info\nTELUS International AI-Data Solu...
242,senior-data-scientist-poatek-brazil-applicants...,2023-07-11,Senior Data Scientist (Poatek- Brazil Applicants),WillowTree,"Agudos, SP",https://br.linkedin.com/jobs/view/senior-data-...,"Senior Data Scientist\n\nSão Paulo-SP, Brazil ..."
308,ml-architect-at-turing-3643189129,2023-07-11,ML Architect,Turing,"Rio de Janeiro, RJ",https://br.linkedin.com/jobs/view/ml-architect...,A rapidly-growing company that is leveraging t...
239,lead-data-scientist-at-turing-3638256512,2023-07-11,Lead Data Scientist,Turing,Brasil,https://br.linkedin.com/jobs/view/lead-data-sc...,A fast-growing company that is developing the ...
141,product-owner-s%C3%AAnior-at-hprojekt-group-36...,2023-07-11,Product Owner - Sênior,Hprojekt Group,"São Paulo, SP",https://br.linkedin.com/jobs/view/product-owne...,Job Description\n\nA Hprojekt é uma consultori...
404,part-time-personalized-internet-ads-assessor-g...,2023-07-11,Part-Time Personalized Internet Ads Assessor -...,TELUS International AI Data Solutions,Brasil,https://br.linkedin.com/jobs/view/part-time-pe...,Company info\nTELUS International AI-Data Solu...
104,chief-technology-officer-gt-school-remote-%248...,2023-07-11,"Chief Technology Officer, gt.school (Remote) -...",Crossover,"Salvador, BA",https://br.linkedin.com/jobs/view/chief-techno...,Crossover is the world's #1 source of full-tim...
61,pessoa-engenheira-de-machine-learning-s%C3%AAn...,2023-07-11,Pessoa Engenheira de Machine Learning Sênior,Serasa Experian,"São Paulo, SP",https://br.linkedin.com/jobs/view/pessoa-engen...,Company Description\n\nAqui na Serasa criamos ...
